In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(temperature=0, model="gpt-4o")

email_conversation = """From: 박지영 (jiyoung.park@techsolutions.kr)
To: 김민수 (minsu@globaltech.com)
Subject: AI 솔루션 도입 제안 및 컨설팅 미팅 요청

김민수 부장님께,

안녕하세요, 테크솔루션즈의 박지영 이사입니다. 귀사의 디지털 전환 계획에 대해 알게 되어 연락드립니다. 저희 테크솔루션즈는 AI 기반 비즈니스 솔루션 분야에서 10년 이상의 경험을 보유한 선도기업입니다.

귀사의 비즈니스 프로세스를 최적화할 수 있는 맞춤형 AI 솔루션을 제안드리고 싶습니다. 특히 데이터 분석, 자동화 시스템, 예측 모델링 분야에서 혁신적인 솔루션을 보유하고 있습니다.

구체적인 논의를 위해 다음 달 3월 20일 오후 2시에 미팅을 제안드립니다. 귀사의 요구사항과 목표를 자세히 파악하고, 최적의 솔루션을 제시해드리고 싶습니다.

검토 부탁드립니다.

감사합니다.

박지영
이사
테크솔루션즈
"""

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출하세요. \n {email_conversation}"
)

chain = prompt | llm | StrOutputParser()

answer = chain.invoke({"email_conversation" : email_conversation})

print(answer)


- 발신자: 박지영, 테크솔루션즈 이사
- 수신자: 김민수, 글로벌테크 부장
- 주제: AI 솔루션 도입 제안 및 컨설팅 미팅 요청
- 주요 내용:
  - 테크솔루션즈는 AI 기반 비즈니스 솔루션 분야에서 10년 이상의 경험을 보유한 선도기업임.
  - 데이터 분석, 자동화 시스템, 예측 모델링 분야에서 맞춤형 AI 솔루션 제안.
  - 3월 20일 오후 2시에 미팅 제안.
  - 귀사의 요구사항과 목표를 파악하여 최적의 솔루션 제시 희망.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

llm = ChatOpenAI(temperature=0, model="gpt-4o")

email_conversation = """From: 박지영 (jiyoung.park@techsolutions.kr)
To: 김민수 (minsu@globaltech.com)
Subject: AI 솔루션 도입 제안 및 컨설팅 미팅 요청

김민수 부장님께,

안녕하세요, 테크솔루션즈의 박지영 이사입니다. 귀사의 디지털 전환 계획에 대해 알게 되어 연락드립니다. 저희 테크솔루션즈는 AI 기반 비즈니스 솔루션 분야에서 10년 이상의 경험을 보유한 선도기업입니다.

귀사의 비즈니스 프로세스를 최적화할 수 있는 맞춤형 AI 솔루션을 제안드리고 싶습니다. 특히 데이터 분석, 자동화 시스템, 예측 모델링 분야에서 혁신적인 솔루션을 보유하고 있습니다.

구체적인 논의를 위해 다음 달 3월 20일 오후 2시에 미팅을 제안드립니다. 귀사의 요구사항과 목표를 자세히 파악하고, 최적의 솔루션을 제시해드리고 싶습니다.

검토 부탁드립니다.

감사합니다.

박지영
이사
테크솔루션즈
"""

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸사람")
    email: str = Field(description="메일을 보낸사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")
    
parser = PydanticOutputParser(pydantic_object=EmailSummary)

print(parser.get_format_instructions())

prompt = PromptTemplate.from_template(
"""
You are helpful assistant. Please answer the following question in KOREAN.

Question:
{question}

EMIAL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

prompt =  prompt.partial(format=parser.get_format_instructions())

chain =  prompt | llm

response = chain.invoke(
    {
        "email_conversation" : email_conversation,
        "question": "이메일 내용중 중요한 내용을 추출해주세요"
    }
)

print(response.content)


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```
```json
{
    "person": "박지영",
    "email": "jiyoung.park@techso

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

llm = ChatOpenAI(temperature=0, model="gpt-4o")

email_conversation = """From: 박지영 (jiyoung.park@techsolutions.kr)
To: 김민수 (minsu@globaltech.com)
Subject: AI 솔루션 도입 제안 및 컨설팅 미팅 요청

김민수 부장님께,

안녕하세요, 테크솔루션즈의 박지영 이사입니다. 귀사의 디지털 전환 계획에 대해 알게 되어 연락드립니다. 저희 테크솔루션즈는 AI 기반 비즈니스 솔루션 분야에서 10년 이상의 경험을 보유한 선도기업입니다.

귀사의 비즈니스 프로세스를 최적화할 수 있는 맞춤형 AI 솔루션을 제안드리고 싶습니다. 특히 데이터 분석, 자동화 시스템, 예측 모델링 분야에서 혁신적인 솔루션을 보유하고 있습니다.

구체적인 논의를 위해 다음 달 3월 20일 오후 2시에 미팅을 제안드립니다. 귀사의 요구사항과 목표를 자세히 파악하고, 최적의 솔루션을 제시해드리고 싶습니다.

검토 부탁드립니다.

감사합니다.

박지영
이사
테크솔루션즈
"""

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸사람")
    email: str = Field(description="메일을 보낸사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")
    
prompt = PromptTemplate.from_template(
"""
You are helpful assistant. Please answer the following question in KOREAN.

Question:
{question}

EMIAL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

prompt =  prompt.partial(format=parser.get_format_instructions())

chain =  prompt | llm

    
parser = PydanticOutputParser(pydantic_object=EmailSummary)

response = chain.invoke(
    {
        "email_conversation" : email_conversation,
        "question": "이메일 내용중 중요한 내용을 추출해주세요"
    }
)

structured_output = parser.parse(response.content)

print(structured_output.person)
print(structured_output.email)
print(structured_output.subject)
print(structured_output.summary)

박지영
jiyoung.park@techsolutions.kr
AI 솔루션 도입 제안 및 컨설팅 미팅 요청
테크솔루션즈의 박지영 이사가 김민수 부장님에게 AI 기반 비즈니스 솔루션을 제안하며, 데이터 분석, 자동화 시스템, 예측 모델링 분야에서의 혁신적인 솔루션을 소개하고자 합니다. 이를 위해 3월 20일 오후 2시에 미팅을 제안합니다.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

llm = ChatOpenAI(temperature=0, model="gpt-4o")

email_conversation = """From: 박지영 (jiyoung.park@techsolutions.kr)
To: 김민수 (minsu@globaltech.com)
Subject: AI 솔루션 도입 제안 및 컨설팅 미팅 요청

김민수 부장님께,

안녕하세요, 테크솔루션즈의 박지영 이사입니다. 귀사의 디지털 전환 계획에 대해 알게 되어 연락드립니다. 저희 테크솔루션즈는 AI 기반 비즈니스 솔루션 분야에서 10년 이상의 경험을 보유한 선도기업입니다.

귀사의 비즈니스 프로세스를 최적화할 수 있는 맞춤형 AI 솔루션을 제안드리고 싶습니다. 특히 데이터 분석, 자동화 시스템, 예측 모델링 분야에서 혁신적인 솔루션을 보유하고 있습니다.

구체적인 논의를 위해 다음 달 3월 20일 오후 2시에 미팅을 제안드립니다. 귀사의 요구사항과 목표를 자세히 파악하고, 최적의 솔루션을 제시해드리고 싶습니다.

검토 부탁드립니다.

감사합니다.

박지영
이사
테크솔루션즈
"""

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸사람")
    email: str = Field(description="메일을 보낸사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")
    
prompt = PromptTemplate.from_template(
"""
You are helpful assistant. Please answer the following question in KOREAN.

Question:
{question}

EMIAL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

parser = PydanticOutputParser(pydantic_object=EmailSummary)

prompt =  prompt.partial(format=parser.get_format_instructions())

chain =  prompt | llm | parser

response = chain.invoke(
    {
        "email_conversation" : email_conversation,
        "question": "이메일 내용중 중요한 내용을 추출해주세요"
    }
)


print(response)


person='박지영' email='jiyoung.park@techsolutions.kr' subject='AI 솔루션 도입 제안 및 컨설팅 미팅 요청' summary='테크솔루션즈의 박지영 이사가 김민수 부장님에게 AI 기반 비즈니스 솔루션을 제안하며, 데이터 분석, 자동화 시스템, 예측 모델링 분야에서의 혁신적인 솔루션을 소개하고자 합니다. 이를 위해 3월 20일 오후 2시에 미팅을 제안합니다.' date='3월 20일 오후 2시'


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

llm = ChatOpenAI(temperature=0, model="gpt-4o")

email_conversation = """From: 박지영 (jiyoung.park@techsolutions.kr)
To: 김민수 (minsu@globaltech.com)
Subject: AI 솔루션 도입 제안 및 컨설팅 미팅 요청

김민수 부장님께,

안녕하세요, 테크솔루션즈의 박지영 이사입니다. 귀사의 디지털 전환 계획에 대해 알게 되어 연락드립니다. 저희 테크솔루션즈는 AI 기반 비즈니스 솔루션 분야에서 10년 이상의 경험을 보유한 선도기업입니다.

귀사의 비즈니스 프로세스를 최적화할 수 있는 맞춤형 AI 솔루션을 제안드리고 싶습니다. 특히 데이터 분석, 자동화 시스템, 예측 모델링 분야에서 혁신적인 솔루션을 보유하고 있습니다.

구체적인 논의를 위해 다음 달 3월 20일 오후 2시에 미팅을 제안드립니다. 귀사의 요구사항과 목표를 자세히 파악하고, 최적의 솔루션을 제시해드리고 싶습니다.

검토 부탁드립니다.

감사합니다.

박지영
이사
테크솔루션즈
"""

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸사람")
    email: str = Field(description="메일을 보낸사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")
    
prompt = PromptTemplate.from_template(
"""
You are helpful assistant. Please answer the following question in KOREAN.

Question:
{question}

EMIAL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

parser = PydanticOutputParser(pydantic_object=EmailSummary)

prompt =  prompt.partial(format=parser.get_format_instructions())

chain =  prompt | llm | parser

response = chain.invoke(
    {
        "email_conversation" : email_conversation,
        "question": "이메일 내용중 중요한 내용을 추출해주세요"
    }
)

llm_with_structured = ChatOpenAI(
    temperature=0,
    model="gpt-4o"
).with_structured_output(EmailSummary)

answer = llm_with_structured.invoke(email_conversation)

answer


In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="List five{subject}. \n {format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

print(prompt.format(subject="대한민국 관광명소"))

List five대한민국 관광명소. 
 Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="List five{subject}. \n {format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)


model = ChatOpenAI(temperature=0, model="gpt-4o")

chain = prompt | model | output_parser

result = chain.invoke({"subject" :"대한민국 관광명소"})

result[1]

'제주도'

In [13]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

response_schema = [
    ResponseSchema(name="answer", description="사용자 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`,`웹사이트주소` 이어야한다"
    )
]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)

format_instructions =  output_parser.get_format_instructions()

print(format_instructions)

prompt = PromptTemplate(
    template= "answer the question as beset as possible \n Fromat \n {format_instructions} \n Qusetion \n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

model = ChatOpenAI(temperature=0, model="gpt-4o")

chain = prompt | model | output_parser

chain.invoke({"question" : "대한민국의 수도는 어디인가요?"})


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자 질문에 대한 답변
	"source": string  // 사용자의 질문에 답하기 위해 사용된 `출처`,`웹사이트주소` 이어야한다
}
```


{'answer': '대한민국의 수도는 서울입니다.', 'source': 'https://ko.wikipedia.org/wiki/서울특별시'}

In [16]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

class Topic(BaseModel):
    description:str = Field(description="주제에 대한 간결한 설명")
    hashags: str = Field(description="해시태그 형식의 키워드 (2개 이상)")
    
output_parser = JsonOutputParser(pydantic_object=Topic)

print(output_parser.get_format_instructions())

question = "점심먹고 졸린 이유를 알려주세요"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한ai어시스턴트입니다 질문에간결하게 대답하세요"),
        ("user","Format:{format_instructions}\nQuestion:\n{question}")
    ]
)

prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

chain = prompt |model|output_parser

answer = chain.invoke({"question":question})

type(answer)

print(answer)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "주제에 대한 간결한 설명", "title": "Description", "type": "string"}, "hashags": {"description": "해시태그 형식의 키워드 (2개 이상)", "title": "Hashags", "type": "string"}}, "required": ["description", "hashags"]}
```
{'description': '점심 식사 후 졸림은 소화 과정에서 혈액이 소화기관으로 집중되면서 뇌로 가는 혈류가 줄어들기 때문입니다.', 'hashags': '#점심식사 #졸림 #소화'}


In [ ]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

class Topic(BaseModel):
    description:str = Field(description="주제에 대한 간결한 설명")
    hashags: str = Field(description="해시태그 형식의 키워드 (2개 이상)")
    
output_parser = JsonOutputParser()

question = "잠에서 깨는 방법에 대해 알려주세요. 방법에 대한 설명은 `descrikption`에 관련키워드는 `hashtags`에 담아주세요"

# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 대답하세요."),
        ("user", "Format: {format_instructions}\nQuestion:\n{question}")
    ]
)

# 지시사항을 프롬프트에 주입
prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

chain = prompt | model | output_parser

response = chain.invoke({"question" : question})

print(response)

{'description': '잠에서 깨는 방법에는 알람 시계 사용, 자연광 노출, 스트레칭, 물 마시기 등이 있습니다.', 'hashtags': ['#알람시계', '#자연광', '#스트레칭', '#물마시기']}


In [21]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = DatetimeOutputParser()

output_parser.format = "%Y-%m-%d"

template = """
Answer the user question:

#Format Instructions:
{format_instructions}

#Question:
{question}

#Answer:
"""

prompt = PromptTemplate.from_template(
    template=template,
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    }
)

chain = prompt | ChatOpenAI(temperature=0, model="gpt-4o") | output_parser

response = chain.invoke({"question": "Apple이 창업한 연도"})

print(response)

print(response.strftime("%Y-%m-%d"))

1976-04-01 00:00:00
1976-04-01


In [24]:
from enum import Enum
from langchain.output_parsers.enum import EnumOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

class Colors(Enum):
    RED="빨간색"
    GREEN="초록색"
    BLUE="파란색"
    
parser = EnumOutputParser(enum=Colors)

prompt = PromptTemplate.from_template(
"""
다음의 물체는 어떤 색깔인가요?

Object : {object}

Instructions: {instructions}
"""
).partial(instructions=parser.get_format_instructions())

chain = prompt | ChatOpenAI() | parser

response = chain.invoke({"object": "사과"})

print(response)

type(response)

print(response.value)


Colors.RED
빨간색
